In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver

# Search movie title on IMDB

In [45]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
# options.experimental_options["prefs"] = {
#     "profile.managed_default_content_settings.images": 2,
#     "profile.managed_default_content_settings.css": 2
# }
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)

url = "https://www.imdb.com/"
driver.get(url)

In [46]:
def find_movie_year(movie_name):
    typetext = wait.until(EC.element_to_be_clickable((By.ID, "suggestion-search")))
    typetext.click()
    typetext.send_keys(movie_name)
    typetext.submit()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    year = soup.find("span", class_="ipc-metadata-list-summary-item__li").text
    try:
        year = int(year)
    except:
        try:
            year = int(year.split("–")[0])
        except:
            year = np.nan
    finally:
        return year

In [48]:
find_movie_year("Oceans")

2004

In [47]:
%%time
pre = pd.read_csv("../data/pre_release.csv")
for movie in pre.head(10)["movie_title"]:
    movie = movie.strip()
    print(f"{movie} was released in {find_movie_year(movie)}")

Miami Vice was released in 1984
The Spanish Prisoner was released in 1997
Conan the Barbarian was released in 1982
The Core was released in 2003
Dredd was released in 2012
Creepshow was released in 2019
Battlefield Earth was released in 2000
Oceans was released in nan
A Sound of Thunder was released in 2005
Top Cat Begins was released in 2015
CPU times: total: 750 ms
Wall time: 24.8 s
